In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import math
import cv2
import librosa 
import librosa.display
import IPython.display as ipd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K

In [ ]:
!pip install -U efficientnet -qq

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
def merge_history(hlist):
    history = {}
    for k in hlist[0].history.keys():
        history[k] = sum([h.history[k] for h in hlist], [])
    return history

def vis_training(h, start=1):
    epoch_range = range(start, len(h['loss'])+1)
    s = slice(start-1, None)

    plt.figure(figsize=[16,4])

    n = int(len(h.keys()) / 2)

    for i in range(n):
        k = list(h.keys())[i]
        plt.subplot(1,n,i+1)
        plt.plot(epoch_range, h[k][s], label='Training')
        plt.plot(epoch_range, h['val_' + k][s], label='Validation')
        plt.xlabel('Epoch'); plt.ylabel(k); plt.title(k)
        plt.grid()
        plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
train = pd.read_csv('../input/g2net-q-transform-69x65/images/training_labels.csv')
print(train.shape, '\n')
train.head()

In [ ]:
SPEC_PATH = '../input/g2net-q-transform-69x65/images'

class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, df, batch_size=32,n_batches = None, img_size=(69,65), shuffle=True, is_train=True):
        self.df = df
        self.n = len(df)
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.is_train = is_train
        self.on_epoch_end()
        self.n_batches = n_batches
        
    def on_epoch_end(self):
        self.indices = np.arange(self.n)
        if self.shuffle == True:
            np.random.shuffle(self.indices)   
    
    def __len__(self):
        if self.n_batches is None:
            return math.ceil( self.n / self.batch_size )
        return self.n_batches
    
    def __getitem__(self, batch_index):
        start = batch_index * self.batch_size
        end = (batch_index + 1) * self.batch_size
        indices = self.indices[start:end]
        
        return self.__data_generation(indices)
    
    def __data_generation(self, batch_indices):
        batch_size = len(batch_indices)
        
        X = np.zeros(shape=(batch_size, self.img_size[0], self.img_size[1], 3))
        y = np.zeros(batch_size)
        
        for i, idx in enumerate(batch_indices):
            ID = self.df.id.values[idx]
            y[i] = self.df.target.values[idx]
            
            SET = 'train' if self.is_train else 'test'
            path = f'{SPEC_PATH}/{SET}/{ID}.npy'
            data_array = np.load(path)
            
            X[i,:,:,:] = data_array
            
        return X, y
    

GENERATOR_TEST = True

if GENERATOR_TEST:
    temp_gen = DataGenerator(train, batch_size=8, shuffle=False)
    X,y = temp_gen.__getitem__(0)

    print(X.shape)
    print(y)

In [ ]:
train_df, valid_df = train_test_split(train, test_size=0.15)
train_loader = DataGenerator(train_df, batch_size=64, shuffle=True)
valid_loader = DataGenerator(valid_df, batch_size=64, shuffle=True) #n_batch= x for lower number of batches

In [ ]:
model = efn.EfficientNetB7(input_shape=(69,65,3), include_top=False, weights='imagenet')



model.trainable = False

In [ ]:
cnn = Sequential([
    model,
    
    GlobalAveragePooling2D(),
    
    #Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    #Dropout(0.1),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

cnn.summary()

In [ ]:
%%time

opt = tf.keras.optimizers.Adam(0.01)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

h1 = cnn.fit(train_loader, epochs=5, validation_data=valid_loader, verbose=1)

In [ ]:
history = merge_history([h1])
vis_training(history)

In [ ]:
model.trainable = True

opt = tf.keras.optimizers.Adam(.0001)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

h2 = cnn.fit(train_loader, epochs=5, validation_data=valid_loader, verbose=1)

In [ ]:
h2.history['auc'] = h2.history['auc_1']
h2.history['val_auc'] = h2.history['val_auc_1']

In [ ]:
history = merge_history([h1,h2])
vis_training(history, start = 5)

In [ ]:
test = pd.read_csv('../input/g2net-q-transform-69x65/images/sample_submission.csv')
print(test.shape)
test.head()

In [ ]:
test_loader = DataGenerator(test, batch_size=64, shuffle=False, is_train=False)

In [ ]:
test_prob = cnn.predict(test_loader)

In [ ]:
cnn.save('PBCQT1.h')

In [ ]:
submission = pd.read_csv('../input/g2net-q-transform-69x65/images/sample_submission.csv')
submission.target = test_prob[:,0]
submission

In [ ]:
submission.to_csv('submissionqformPBCQT.csv', header=True, index= False)